Welcome to the example code to use PIXIS camera with the Monochromator. 
We first being with importing relevant python pachanges and the API for the PIXIS Camera: 

In [2]:
import pandas as pd
import numpy as np
import monochromatorapi as mcapi # Import monochromator api
from sophiaapi import *
from picoapi import *

folder = '20240626'

MCPort = 'COM4'
ShutPort = 'COM6'

Ch1ON=1
Ch2ON=1
nsamples=30
interval=0.1
asrl="asrl5::instr" #port for the picoammeter
# BP Picoameter settings

picoa=picoammeter_initialize(Ch1ON,Ch2ON,interval,nsamples,asrl,debug=False)
# BP Initialize Picoameter.

samples will be taken in a 3.0 second exposure
Sent *IDN?
Received KEITHLEY INSTRUMENTS INC.,MODEL 6482,4446862,A02   Jan 18 2017 10:20:01/A02  /F
KEITHLEY INSTRUMENTS INC.,MODEL 6482,4446862,A02   Jan 18 2017 10:20:01/A02  /F
added to output queue
Sent *RST
Sent *OPC?
Received 1
Sent :SENS1:CURR:RANG:AUTO ON
Sent :SENS1:CURR:RANG:AUTO:ULIM 2.1e-08
Sent :SENS1:CURR:RANG:AUTO:LLIM 2e-09
Sent :SENS1:CURR:NPLC 10.0
Sent :SOUR1:VOLT:RANGE:AUTO 1 
Sent :SOUR1:VOLT 0
Sent OUTP1 ON
Sent :SENS2:CURR:RANG:AUTO ON
Sent :SENS2:CURR:RANG:AUTO:ULIM 2.1e-08
Sent :SENS2:CURR:RANG:AUTO:LLIM 2e-09
Sent :SENS2:CURR:NPLC 10.0
Sent :SOUR2:VOLT:RANGE:AUTO 1 
Sent :SOUR2:VOLT 0
Sent OUTP2 ON


In [3]:
experiment_sophia = 'SOPHIA_MC'
status, auto, experiment = pixis_load_experiment(experiment_sophia)

if status==True: 
    print("SOPHIA is ready")

Connection successful
SOPHIA is ready


In [3]:
pixis_set_temperature(-35.0)

KeyboardInterrupt: 

In [4]:
pixis_set_value(ExperimentSettings.OnlineExportEnabled,True) #enables export of acquired data other than the native *.spe format
pixis_set_value(ExperimentSettings.OnlineExportFormat, AddIns.ExportFileType(1)) #  1 for FITS format
pixis_set_value(ExperimentSettings.OnlineExportOutputOptionsExportedFilesOnly,True)
pixis_set_value(ExperimentSettings.OnlineExportFormatOptionsIncludeExperimentInformation,True)
exp_directory=pixis_set_folder(folder,'D:\\Sophia')
pixis_set_value(ExperimentSettings.FileNameGenerationAttachDate,False)
pixis_set_value(ExperimentSettings.FileNameGenerationAttachTime,False)
pixis_set_value(ExperimentSettings.FileNameGenerationAttachIncrement,False)

Created empty folder D:\Sophia\20240626 for storing experiment files
Custom directory set to D:\Sophia\20240626 


In [5]:
start_wl=200.0 #nm
end_wl=600.0 #nm
wl_step=10.0 #nm 
exp_time=10000.0 #milliseconds 

mcapi.go_to_fromhome(MCPort, start_wl)

pixis_set_value(CameraSettings.HardwareIOOutputSignal, AddIns.OutputSignal(5))
pixis_set_value(CameraSettings.HardwareIOInvertOutputSignal, False)
data = picoa_get_measurement(picoa,'D:/Sophia/{}/picoa_{:.0f}.csv'.format(folder, 0),interval,nsamples)
pixis_set_value(CameraSettings.HardwareIOInvertOutputSignal, True)

for current_wl in np.arange(start_wl, end_wl+wl_step, wl_step):
    print('Taking Bias')
    pixis_set_value(ExperimentSettings.FileNameGenerationBaseFileName, 'bias_{:.0f}_1'.format(current_wl))
    pixis_take_bias_frames(1)
    pixis_set_value(ExperimentSettings.FileNameGenerationBaseFileName, 'bias_{:.0f}_2'.format(current_wl))
    pixis_take_bias_frames(1)

    print('Taking Dark')
    pixis_set_value(ExperimentSettings.FileNameGenerationBaseFileName, 'dark_{:.0f}_1'.format(current_wl))
    pixis_take_dark_frames(1, exp_time)
    pixis_set_value(ExperimentSettings.FileNameGenerationBaseFileName, 'dark_{:.0f}_2'.format(current_wl))
    pixis_take_dark_frames(1, exp_time)
    
    print('Taking Exposure')
    pixis_set_value(ExperimentSettings.FileNameGenerationBaseFileName, 'science_{:.0f}_1'.format(current_wl))
    pixis_take_exposure(exp_time)
    time.sleep(exp_time/1000)
    print('Taking Exposure 2')
    pixis_set_value(ExperimentSettings.FileNameGenerationBaseFileName, 'science_{:.0f}_2'.format(current_wl))
    pixis_take_exposure(exp_time)

    pixis_set_value(CameraSettings.HardwareIOOutputSignal, AddIns.OutputSignal(5))
    pixis_set_value(CameraSettings.HardwareIOInvertOutputSignal, True)
    data = picoa_get_measurement(picoa,'D:/Sophia/{}/picoa_{:.0f}.csv'.format(folder, current_wl),interval,nsamples)
    pixis_set_value(CameraSettings.HardwareIOInvertOutputSignal, False)

    print('Moving MC')
    mcapi.go_to_from(MCPort,current_wl,current_wl+wl_step)

print('Finished, homing MC')
mcapi.home(MCPort)
print("Scan complete")
# pixis_set_temperature(23)
# pixis_get_current_temperature()

home circuit enabled, prepared to home
scanner is below home so moving up to home
increasing wavelength at a rate of 23KHz
Limits status is:34
Limits status is:34
Limits status is:34
Limits status is:2
scan controller stopped
decrease wavelength for 3 revolutions
increase wavelength for 2 revolutions
high accuracy circuit enabled
finding edge of home flag at 4500KHz, this will take about 12 seconds
scan controller stopped
homing movement successful
disabled home circuit
homing successful
scan controller is moving for -221.0nm
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33
moving status: 33


KeyboardInterrupt: 